# Importação de bibliotecas

In [150]:
import pandas as pd
from scipy.stats import normaltest
from statsmodels.stats.weightstats import DescrStatsW
from statsmodels.stats.weightstats import CompareMeans
from scipy.stats import wilcoxon, ranksums

# Carregamento dos dados

In [29]:
amostras = pd.read_csv('/Users/mikeiasoliveira/Documents/Projetos/análise_dados_enem/analise-dados-enem/dados/amostras/amostra-redacao.csv', sep =';')

# Limpeza dos dados

In [32]:
amostras = amostras.dropna(subset= 'NU_NOTA_REDACAO')

# Testes estatísticos

----
----

## Teste de normalidade

In [39]:
stats_normTestRedacao, pValor_normTestRedacao = normaltest(amostras.NU_NOTA_REDACAO)
print(f'O P-value é igual a: {round(pValor_normTestRedacao, 2)}')


O P-value é igual a: 0.06


$ H_0: $ A distribuição provém de uma distribuição normal <br>
$ H_1: $ A distribuição não provém de uma distribuição normal <br>
$ p-value > \alpha $ <br><br>

<font color='green'>A amostra provém de uma distribuição normal</font>


----

In [41]:
amostras.columns

Index(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
       'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA',
       'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH',
       'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TP_STATUS_REDACAO',
       'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004',

## Análise por raça

Temos que:

Código|Raça|
------|----|
0|Não declarado
1|Branca
2|Preta
3|Parda
4|Amarela
5|Indígena 

In [44]:
amostras.TP_COR_RACA.replace({0:'Não declarado', 1:'Branca', 2:'Preta', 3:'Parda', 4:'Amarela', 5:'Indígena'}, inplace = True)

### Pessoas Brancas e pessoas pretas

$ H_0: $ A nota da média redação de pretos é igual a de brancos <br>
$ H_1: $ A nota da média redação de pretos não é igual a de brancos <br>

In [ ]:
amostras_pessoas_pretas = amostras.query("TP_COR_RACA == 'Preta'")['NU_NOTA_REDACAO']
amostras_pessoas_brancas = amostras.query("TP_COR_RACA == 'Branca'")['NU_NOTA_REDACAO']

teste_pretos = DescrStatsW(amostras_pessoas_pretas)
teste_brancos = DescrStatsW(amostras_pessoas_brancas)


teste = teste_brancos.get_compare(teste_pretos)

teste.ztest_ind(alternative = 'two-sided')

$ p-value > \alpha $, portando não podemos descartar $H_0$ <br><br>


<font color='green'>A nota da média redação de pretos é igual a de brancos</font>

---

### Pessoas Indigenas e pessoas brancas

$ H_0: $ A nota da média redação de indigenas é igual a de brancos <br>
$ H_1: $ A nota da média redação de indigenas não é igual a de brancos <br>

In [88]:
amostras_pessoas_indigena = amostras.query("TP_COR_RACA == 'Indígena'")['NU_NOTA_REDACAO']
amostras_pessoas_brancas = amostras.query("TP_COR_RACA == 'Branca'")['NU_NOTA_REDACAO']

In [89]:
teste_indigenas = DescrStatsW(amostras_pessoas_indigena)
teste_brancos = DescrStatsW(amostras_pessoas_brancas)


teste = teste_brancos.get_compare(teste_indigenas)

In [90]:
teste.ztest_ind(alternative = 'two-sided')

(1.8370386404214292, 0.06620419127361056)

$ p-value > \alpha $, portando não podemos descartar $H_0$ <br><br>


<font color='green'>A nota da média redação de indigenas é igual a de brancos</font>

### Automatizando teste estatístico

In [125]:
racas_1 = list(amostras.TP_COR_RACA.unique())
racas_2 = list(amostras.TP_COR_RACA.unique())

raca_pd_1 = []
raca_pd_2 = []
p_values = []
hipotese = []

validacao = []

for i in racas_1:
    for j in racas_2:
        if i != j:


            amostras_i = amostras.query(f"TP_COR_RACA == '{i}'")['NU_NOTA_REDACAO']
            amostras_j = amostras.query(f"TP_COR_RACA == '{j}'")['NU_NOTA_REDACAO']
                    
            teste_i = DescrStatsW(amostras_i)
            teste_j = DescrStatsW(amostras_j)
                    

            raca_pd_1.append(i)
            raca_pd_2.append(j)

            teste = teste_i.get_compare(teste_j)
            _, p_value = teste.ztest_ind(alternative = 'two-sided')
            p_values.append(p_value)

            if p_value > 0.05:
                hipotese.append('Aceito')
            else:
                hipotese.append('Rejeitado')
    racas_2.remove(i)

pd.DataFrame({'raca-1': raca_pd_1, 'raca-2': raca_pd_2, 'p_value':p_values, 'hipotese':hipotese})


,raca-1,raca-2,p_value,hipotese
0,Branca,Preta,0.817190,Aceito
1,Branca,Não declarado,0.207221,Aceito
2,Branca,Parda,0.169320,Aceito
3,Branca,Amarela,0.987632,Aceito
4,Branca,Indígena,0.066204,Aceito
5,Preta,Não declarado,0.161837,Aceito
6,Preta,Parda,0.146493,Aceito
7,Preta,Amarela,0.941676,Aceito
8,Preta,Indígena,0.029302,Rejeitado
9,Não declarado,Parda,0.052861,Aceito


#### Através dos teste de hipóteses e considerando um nível de confiança de 95%, tivemos que:

$H_0$: A nota da média redação de Não declarado	é igual a de Indígena <br> <font color='red'>Rejeitado</font> <br><br>
$H_0$: A nota da média redação de pessoas Pretas é igual a de Indígena <br> <font color='red'>Rejeitado</font><br><br>

### Aprofundando nas análises a decorrer das raças

#### Pessoas pretas e pessoas indígenas

Vamos analisar a seguinte hipotese:

$H_0$: A nota da redação de pessoas pretas é maior que a nota da redação de pessoas indígenas <br>
$H_1$: A nota da redação de pessoas pretas é maior que a nota da redação de pessoas indígenas

In [164]:
amostras_n_declarado = amostras.query("TP_COR_RACA == 'Não declarado'")['NU_NOTA_REDACAO']
amostras_pessoas_indigenas = amostras.query("TP_COR_RACA == 'Indígena'")['NU_NOTA_REDACAO']

ranksums(amostras_n_declarado, amostras_pessoas_indigenas, alternative='less')

RanksumsResult(statistic=1.9639610121239315, pvalue=0.9752326932821866)

$ p-value > \alpha $, portando podemos descartar $H_0$ <br><br>


<font color='green'>A nota da média redação de pretos é maior que a de Indígena</font>

#### Não declarados e indígenas

$ H_0: $ A nota da média redação de pretos é igual a de brancos <br>
$ H_1: $ A nota da média redação de pretos não é igual a de brancos <br>

In [168]:
amostras_Preta = amostras.query("TP_COR_RACA == 'Preta'")['NU_NOTA_REDACAO']
amostras_pessoas_indigenas = amostras.query("TP_COR_RACA == 'Indígena'")['NU_NOTA_REDACAO']

ranksums(amostras_Preta, amostras_pessoas_indigenas, alternative='less')

RanksumsResult(statistic=1.6090183629051282, pvalue=0.9461938367850651)

In [154]:
ranksums(amostras_n_declarado, amostras_pessoas_indigenas, alternative='greater')

RanksumsResult(statistic=1.9639610121239315, pvalue=0.024767306717813353)

$ p-value > \alpha $, portando não podemos descartar $H_0$ <br><br>


<font color='green'>A nota da média redação de Não declarados é maior que a de Indígena</font>